In [1]:
import tensorflow as tf
from PIL import Image
import os
import numpy as np
from search_videos_in_directory import search_videos
from tensorflow.keras.preprocessing.image import ImageDataGenerator

[INFO] loading model...


In [ ]:
input_direc = input("Enter the absolute path of the input directory: ")
output_direc = input("Enter the absolute path of the output directory: ")
try:
    os.mkdir(output_direc)
except FileExistsError:
    pass
search_videos(input_direc, output_direc)

In [ ]:
for root, dirs, files in os.walk(output_direc):
    for file in files:

        try:
            im = Image.open(root + '/' + str(file))
            im.verify()
        except:
            print(str(file))
            os.remove(root + '/' + str(file))

In [11]:
input_direc = '/home/debanik/downloaded_videos'
output_direc = '/home/debanik/PycharmProjects/Detecting-Deepfakes/Face_images_Deepfakes_only_100'

In [12]:
# create generator
datagen = ImageDataGenerator(samplewise_center=True, 
                             samplewise_std_normalization=True, 
                             rotation_range=10, 
                             horizontal_flip = True, 
                             validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(output_direc, class_mode='binary', batch_size=64)
# # confirm the iterator works
# batchX, batchy = train_it.next()
# print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 199168 images belonging to 2 classes.


In [4]:
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16

In [7]:
x = Input(shape = (256, 256, 3))

#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16 = VGG16(include_top=False)

#Use the generated model 
output_vgg16 = model_vgg16(x)

#Add the fully-connected layers 
y = Flatten()(output_vgg16)
y = Dense(1000, activation = 'relu')(y)
y = Dense(1000, activation = 'relu')(y)
y = Dropout(0.5)(y)
y = Dense(1, activation='sigmoid')(y)

In [8]:
model = Model(inputs = x, outputs = y)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense (Dense)                (None, 1000)              32769000  
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001  

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit_generator(train_it, epochs = 10)

Epoch 1/10
   2/3112 [..............................] - ETA: 43:34:15 - loss: 2.6706 - acc: 0.4922

KeyboardInterrupt: 